## Unit 4 lesson 4 word2vec

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
import gensim
from gensim.models import word2vec
from spacy.lang.en.examples import sentences

C:\Users\blindside\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\blindside\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
C:\Users\blindside\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\blindside\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\blindside\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


## Data

#### The course said to use three books combined before applying spacy and I ran into problems with that going over my limit, I chose two different books  by chesterton that is under the limit instead

In [12]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text


# Import all the Austen in the Project Gutenberg corpus.
chesterton = ""
for novel in ['brown','ball']:
    work = gutenberg.raw('chesterton-' + novel + '.txt')
    chesterton = chesterton + work

# Clean the data.
chesterton_clean = text_cleaner(chesterton)

In [13]:
# Parse the data. This can take some time.
nlp = spacy.load('en')
chesterton_doc = nlp(chesterton_clean)

In [14]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences = []
for sentence in chesterton_doc.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)


print(sentences[20])
print('We have {} sentences and {} tokens.'.format(len(sentences), len(chesterton_clean)))

['-pron-', 'rich', 'confusion', 'social', 'self', 'congratulation', 'bodily', 'disarray']
We have 8373 sentences and 849431 tokens.


In [15]:
model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [17]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('colour', 0.998045802116394), ('wind', 0.9979996681213379), ('strike', 0.9978877305984497), ('roman', 0.9976905584335327), ('vessel', 0.9975874423980713), ('firm', 0.9975014328956604), ('smooth', 0.9973048567771912), ('pain', 0.9972853660583496), ('able', 0.9972162246704102), ('somewhat', 0.9971903562545776)]
0.9793983008393077


C:\Users\blindside\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  # This is added back by InteractiveShellApp.init_path()


lunch


## Drill 0 

Take a few minutes to modify the hyperparameters of this model and see how its answers change. Can you wrangle any improvements?

In [18]:
model = word2vec.Word2Vec(
    sentences,
    workers=8,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=8,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=450,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [19]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('save', 0.9984132051467896), ('ring', 0.9982326030731201), ('begin', 0.9981318712234497), ('take', 0.9980866312980652), ('particular', 0.9980510473251343), ('forward', 0.9979133605957031), ('gold', 0.9978599548339844), ('pull', 0.9978576302528381), ('island', 0.9978042244911194), ('cross', 0.9976385235786438)]
0.9858593541305107


C:\Users\blindside\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  # This is added back by InteractiveShellApp.init_path()


lunch


This model is pretty bad, none of these words are close to lady and man and lunch was not the correct choice.

In [22]:
model = word2vec.Word2Vec(
    sentences,
    workers=8,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=8,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=1000,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [23]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('bronze', 0.9992465972900391), ('receive', 0.9992438554763794), ('costume', 0.9992180466651917), ('flee', 0.9992027282714844), ('powerful', 0.99919593334198), ('near', 0.9991490840911865), ('assent', 0.9990918636322021), ('prisoner', 0.9990500211715698), ('character', 0.9990233182907104), ('brain', 0.999003529548645)]
0.9831601507335278


C:\Users\blindside\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  # This is added back by InteractiveShellApp.init_path()


dinner


No improvements, I believe the best bet would be to use a different dataset or add more books

## Drill 1: Word2Vec on 100B+ words

In [29]:
model = gensim.models.KeyedVectors.load_word2vec_format ('GoogleNews-vectors-negative300.bin', binary=True)

In [31]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

C:\Users\blindside\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\blindside\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


[('fella', 0.6031545400619507), ('gentleman', 0.5849651098251343), ('chap', 0.5543248653411865), ('gent', 0.543907880783081), ('guy', 0.5265034437179565), ('lad', 0.5139425992965698), ('feller', 0.5072450041770935), ('bloke', 0.49030163884162903), ('rascal', 0.48736995458602905), ('ladies', 0.47617608308792114)]


C:\Users\blindside\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


0.6609882886573529
marriage


## Conclusion

The 3Gb file from google faired much better than my first model and even picked the correct choice between marriage and mealtimes. The model was able to pick out similarities with lady and man that were all accurate, with one exception being rascal. The google model seems very powerful and I would like to use it on other datasets in the future.